In [1]:
import os

# Check the updated current working directory
current_directory = os.getcwd()

print(current_directory)


C:\Users\chitr


In [4]:
from IPython.core.compilerop import contextmanager
!pip install pulp
import pulp
import pandas as pd

In [5]:
# Change the current working directory
new_directory = "C:/Users/chitr/OneDrive/Uddhav Working/Germany/OVGU/Sem-3/Scientific Project/Common/Codes/PFS"
os.chdir(new_directory)
# Read the Excel file
coverage_df = pd.read_excel("coverage_matrix.xlsx", header=None)
population_df = pd.read_excel("PFS_population_weights.xlsx", header=None)

In [6]:
# Convert the DataFrames to matrices
Coverage = coverage_df.values.tolist()
population = population_df.values.flatten().tolist()

In [7]:
# Constants
num_circles = 11
max_stations = 3

In [31]:
# Set up the problem
prob = pulp.LpProblem("Population_Coverage_Problem", pulp.LpMaximize)

# Decision variables
stations = pulp.LpVariable.dicts("Station", range(num_circles), cat="Binary")

# Objective function: maximize the total population covered
prob += pulp.lpSum(population[i] * stations[i] for i in range(num_circles))

# Constraints
# Ensure 100% coverage
for i in range(num_circles):
    prob += (
        pulp.lpSum(
            stations[j]
            for j in range(num_circles)
            if Coverage[i][j] == 1
        )
        >= 1
    )


In [32]:
# Limit the number of selected circles
prob += pulp.lpSum(stations) <= max_stations

# Solve the problem
prob.solve()

1

In [33]:
# Print the optimal solution
print("Optimal Solution:")
selected_circles = [i + 1 for i in range(num_circles) if pulp.value(stations[i]) == 1]
print("Selected Circles:", selected_circles)

Optimal Solution:
Selected Circles: [2, 8, 10]


In [28]:
#sensitivity Analysis
# Loop through different maximum number of stations
for max_stations_new in range(1, 4):
    # Set up the problem
    prob = pulp.LpProblem("Population_Coverage_Problem", pulp.LpMaximize)

    # Decision variables
    stations = pulp.LpVariable.dicts("Station", range(num_circles), cat="Binary")

    # Objective function: maximize the total population covered
    prob += pulp.lpSum(population[i] * stations[i] for i in range(num_circles))

    # Constraints
    # Ensure 100% coverage
    for i in range(num_circles):
        prob += (
            pulp.lpSum(
                stations[j]
                for j in range(num_circles)
                if Coverage[i][j] <= 1
            )
            >= 1
        )
    # Limit the number of selected circles
    prob += pulp.lpSum(stations) <= max_stations_new

    # Solve the problem
    prob.solve()
    # Print the optimal solution
    print(f"Max Stations: {max_stations_new}")
    selected_circles = [i + 1 for i in range(num_circles) if pulp.value(stations[i]) == 1]
    print("Selected Circles:", selected_circles)
    print("Total Population Covered:", pulp.value(prob.objective))
    print("-" * 50)   

Max Stations: 1
Selected Circles: [5]
Total Population Covered: 68011.0
--------------------------------------------------
Max Stations: 2
Selected Circles: [5, 8]
Total Population Covered: 107980.0
--------------------------------------------------
Max Stations: 3
Selected Circles: [4, 5, 8]
Total Population Covered: 144489.0
--------------------------------------------------
